# Laboratorium 1 - analiza koszykowa

## Przygotowanie

 * pobierz i wypakuj dataset: https://kaggle.com/datasets/rashikrahmanpritom/groceries-dataset-for-market-basket-analysismba?resource=download&select=basket.csv
   * alternatywnie, pobierz plik `basket.csv` z Teamsów
 * [opcjonalnie] utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install more-itertools`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety
from more_itertools import powerset

In [2]:
# definiujemy stałe
PATH = './basket.csv'
EPSILON = 0.001
K = 4

In [3]:
# wczytujemy dane o koszykach
def read_baskets(path: str) -> list[tuple[str]]:
    with open(path) as f:
        raw = f.read()
    
    baskets = [set([y.lower() for y in x.split(',') if y]) for x in raw.split('\n')[1:] if x]

    return baskets


def unique_products(baskets: list[tuple[str]]) -> list[str]:
    products = set()

    for basket in baskets:
        products.update(basket)
    
    return sorted(list(products))


baskets = read_baskets(PATH)
products = unique_products(baskets)

## Część 2. - obliczanie wskaźników

In [4]:
# obliczamy strukturę danych (np. słownik albo graf) przechowującą
# wszystkie interesujące nas wartości `support`
def get_supports(baskets: list[tuple[str]], all_products: list[str], epsilon: float):
    supports = {}
    baskets_ids_dict = {}
    n = len(baskets)
    singles = []

    for product in all_products:
        numerator = 0
        baskets_ids = []

        for i, basket in enumerate(baskets):
            if product in basket:
                numerator += 1
                baskets_ids.append(i)

        if numerator / n >= epsilon:
            singles.append((product,))
            supports[(product,)] = numerator / n
            baskets_ids_dict[(product,)] = baskets_ids

    prev = singles

    for _ in range(2, K + 1):
        curr = []

        for s in singles:
            for p in prev:
                if s[0] not in p:
                    c = tuple(sorted(p + s))
                    numerator = 0

                    for basket_id in baskets_ids_dict[s]:
                        if all(item in baskets[basket_id] for item in p):
                            numerator += 1
                    
                    if numerator / n >= epsilon:
                        curr.append(c)
                        supports[c] = numerator / n

        prev = curr
    
    return supports


supports = get_supports(baskets, products, EPSILON)
supports

{('abrasive cleaner',): 0.0014702933903628951,
 ('artif. sweetener',): 0.0019381140145692708,
 ('baking powder',): 0.008086613646995923,
 ('bathroom cleaner',): 0.0011361358016440553,
 ('beef',): 0.03395041101383412,
 ('berries',): 0.021787074784468355,
 ('beverages',): 0.016574216400454454,
 ('bottled beer',): 0.04531176903027468,
 ('bottled water',): 0.06068301811134131,
 ('brandy',): 0.0025395976742631824,
 ('brown bread',): 0.03762614448974136,
 ('butter',): 0.03522020985096572,
 ('butter milk',): 0.017576689166610975,
 ('cake bar',): 0.006148499632426653,
 ('candles',): 0.004410880171088686,
 ('candy',): 0.014368776314910112,
 ('canned beer',): 0.04691572545612511,
 ('canned fish',): 0.007685624540533315,
 ('canned fruit',): 0.001403461872619127,
 ('canned vegetables',): 0.005480184454988973,
 ('cat food',): 0.011829178640646929,
 ('cereals',): 0.002806923745238254,
 ('chewing gum',): 0.012029673193878232,
 ('chicken',): 0.027868742899151238,
 ('chocolate',): 0.02359152576355009,


In [5]:
# definiujemy funkcje obliczające support, confidence i lift
def support(supports, products: tuple[str]) -> float:
    ps = tuple(sorted(products))

    return supports[ps] if ps in supports else 0.


def confidence(supports, prior_products: tuple[str], following_products: tuple[str]) -> float:
    numerator = support(supports, tuple(set(prior_products + following_products)))
    denominator = support(supports, prior_products)

    return 0. if numerator * denominator == 0. else numerator / denominator


def lift(supports, prior_products: tuple[str], following_products: tuple[str]) -> float:
    numerator = support(supports, tuple(set(prior_products + following_products)))
    denominator = support(supports, prior_products) * support(supports, following_products)

    return 0. if numerator * denominator == 0. else numerator / denominator

In [6]:
print(support(supports, ('whole milk', 'rolls/buns')))
print(confidence(supports, ('whole milk', 'rolls/buns'), ('yogurt',)))
print(lift(supports, ('whole milk', 'rolls/buns'), ('yogurt',)))

0.013967787208447505
0.09569377990430622
1.1142926293448512


## Część 3. - generowanie rekomendacji

In [7]:
# wyznaczamy listę potencjalnych rekomendacji
# rekomendowane artykuły powinny mieć lift > 1 i jak najwyższy confidence
def generate_basic_candidates(
    basket: tuple[str], products: list[str], supports
) -> list[tuple[str, tuple[str], float, float]]:
    # return [(item, subbasket, confidence, lift)]
    candidates = {}

    for prod in products:
        for subbasket in powerset(basket):
            if support(supports, subbasket) and support(supports, (prod,)) and prod not in basket:
                conf = confidence(supports, subbasket, (prod,))
                l = lift(supports, subbasket, (prod,))

                if l > 1:
                    candidates[(prod, subbasket)] = (prod, subbasket, conf, l)

    candidates_list = candidates.values()

    return sorted(candidates_list, key=lambda x: x[2], reverse=True)[:min(5, len(candidates_list))]

In [8]:
# zaproponuj drugi, bardziej zaawansowany algorytm, np.:
# - jeśli produkt X występuje w liście kandydatow kilkukrotnie, oblicz średnią lub iloczyn confidence
# - posortuj kandydatów po iloczynie confidence i lift
def generate_advanced_candidates(
    basket: tuple[str], products: list[str], supports
) -> list[tuple[str, tuple[str], float, float]]:
    # return [(item, subbasket, confidence, lift)]
    candidates = {}

    for prod in products:
        for subbasket in powerset(basket):
            if support(supports, subbasket) and support(supports, (prod,)) and prod not in basket:
                conf = confidence(supports, subbasket, (prod,))
                l = lift(supports, subbasket, (prod,))

                if l > 1:
                    if prod not in candidates:
                        candidates[prod] = (prod, [subbasket], conf, l, 1)
                    else:
                        _, subbaskets, conf_old, l_old, counter = candidates[prod]
                        subbaskets.append(subbasket)
                        conf = (counter * conf_old + conf) / (counter + 1)
                        l = (counter * l_old + l) / (counter + 1)
                        candidates[prod] = (prod, subbaskets, conf, l, counter + 1)

    candidates_list = candidates.values()
    candidates_sorted = sorted(candidates_list, key=lambda x: x[2], reverse=True)
    candidates_top5 = candidates_sorted[:min(5, len(candidates_list))]

    return [(c[0], c[1], c[2], c[3]) for c in candidates_top5]

In [9]:
print(baskets[1])
generate_basic_candidates(baskets[1], products, supports)

{'yogurt', 'whole milk', 'semi-finished bread', 'sausage'}


[('rolls/buns',
  ('whole milk', 'sausage'),
  0.12686567164179105,
  1.1532752398396837),
 ('rolls/buns',
  ('yogurt', 'whole milk'),
  0.11976047904191618,
  1.0886853267947703),
 ('soda', ('whole milk', 'sausage'), 0.11940298507462688, 1.2296124333596987),
 ('soda', ('sausage',), 0.09856035437430785, 1.0149749363405152),
 ('bottled beer', ('sausage',), 0.05537098560354374, 1.2220000849348451)]

In [10]:
print(baskets[1])
generate_advanced_candidates(baskets[1], products, supports)

{'yogurt', 'whole milk', 'semi-finished bread', 'sausage'}


[('rolls/buns',
  [('yogurt', 'whole milk'), ('whole milk', 'sausage')],
  0.12331307534185362,
  1.1209802833172269),
 ('soda',
  [('sausage',), ('whole milk', 'sausage')],
  0.10898166972446736,
  1.1222936848501068),
 ('bottled beer', [('sausage',)], 0.05537098560354374, 1.2220000849348451),
 ('citrus fruit', [('yogurt',)], 0.053696498054474705, 1.0106423904265471),
 ('pastry', [('sausage',)], 0.05315614617940199, 1.0276168156103256)]

In [11]:
print(baskets[33])
generate_basic_candidates(baskets[33], products, supports)

{'white wine', 'yogurt', 'tropical fruit', 'root vegetables', 'soda', 'domestic eggs', 'photo/film'}


[('sausage', ('yogurt',), 0.0669260700389105, 1.1089864739670185),
 ('sausage', ('soda',), 0.06125258086717137, 1.0149749363405152),
 ('citrus fruit', ('yogurt',), 0.053696498054474705, 1.0106423904265471),
 ('shopping bags',
  ('root vegetables',),
  0.04803073967339097,
  1.0093875810856026),
 ('newspapers', ('domestic eggs',), 0.04144144144144145, 1.0654437943097737)]

In [12]:
print(baskets[33])
generate_advanced_candidates(baskets[33], products, supports)

{'white wine', 'yogurt', 'tropical fruit', 'root vegetables', 'soda', 'domestic eggs', 'photo/film'}


[('sausage',
  [('yogurt',), ('soda',)],
  0.06408932545304094,
  1.0619807051537669),
 ('citrus fruit', [('yogurt',)], 0.053696498054474705, 1.0106423904265471),
 ('shopping bags',
  [('root vegetables',)],
  0.04803073967339097,
  1.0093875810856026),
 ('newspapers', [('domestic eggs',)], 0.04144144144144145, 1.0654437943097737),
 ('coffee', [('domestic eggs',)], 0.03783783783783784, 1.1969716016227643)]